In [1]:
from opensearchpy import Field, Boolean, Float, Integer, Document, Keyword, Text, DenseVector, Nested, Date, Object
from opensearchpy import OpenSearch
import numpy as np
import pandas as pd
import datetime

In [107]:
header = ['userId', 'movieId', 'rating', 'timestamp']
df_movies = pd.read_csv('./ml-100k/u.item', sep='|', names=['id', 'name', 'fecha', 'x', 'url'] + list(range(19)) , encoding='latin-1')
df_users = pd.read_csv('./ml-100k/u.user', sep='|',  names=['id', 'gender', 'ocupation', 'zipcode'], encoding='latin-1')

In [108]:
header = ['id', 'name', 'fecha', 'x',
              'url', 'unknown', 'Action', 'Adventure', 'Animation',
              "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
              'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
              'Thriller', 'War', 'Western']
df_movies = pd.read_csv('./ml-100k/u.item', sep='|', encoding = "ISO-8859-1", names=header)
df_movies.head()

,id,name,fecha,x,url,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [109]:
pd.DataFrame(df_users).describe()

,id
count,943.000000
mean,34.051962
std,12.192740
min,7.000000
25%,25.000000
50%,31.000000
75%,43.000000
max,73.000000


In [110]:
pd.DataFrame(df_movies).describe()

,id,x,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
count,1682.000000,0.0,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,...,1682.00000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000
mean,841.500000,NaN,0.001189,0.149227,0.080262,0.024970,0.072533,0.300238,0.064804,0.029727,...,0.01308,0.014269,0.054697,0.033294,0.036266,0.146849,0.060048,0.149227,0.042212,0.016052
std,485.695893,NaN,0.034473,0.356418,0.271779,0.156081,0.259445,0.458498,0.246253,0.169882,...,0.11365,0.118632,0.227455,0.179456,0.187008,0.354061,0.237646,0.356418,0.201131,0.125714
min,1.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,421.250000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,841.500000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1261.750000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1682.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [111]:
df_movies.columns

Index(['id', 'name', 'fecha', 'x', 'url', 'unknown', 'Action', 'Adventure',
       'Animation', 'Children's', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western'],
      dtype='object')

In [112]:
movie_embeddings_matrix = np.load('./vectors/movie_embeddings_matrix.npy')
user_embeddings_matrix = np.load('./vectors/user_embeddings_matrix.npy')
user2Idx = np.load('./vectors/user2Idx.npy', allow_pickle=True).item()
movie2Idx = np.load('./vectors/movie2Idx.npy', allow_pickle=True).item()

In [113]:
len(movie2Idx)

1682

In [114]:
len(user2Idx)

943

In [115]:
df_users['userIdx'] = df_users['id'].apply(lambda x: user2Idx[x])
df_movies['movieIdx'] = df_movies['id'].apply(lambda x: movie2Idx[x])

In [121]:
df_users.head()
df_movies

,id,name,fecha,x,url,unknown,Action,Adventure,Animation,Children's,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movieIdx
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,25
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,148
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,234
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,48
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1645
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1633
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1652
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1668


In [118]:
df_movies.loc[df_movies['url'].isna(), 'url'] = ''
#len(df_movies)

In [119]:
df_movies['url'].isna().sum()

0

In [122]:
df_movies

,id,name,fecha,x,url,unknown,Action,Adventure,Animation,Children's,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movieIdx
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,25
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,148
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,234
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,48
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1645
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1633
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1652
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1668


In [225]:
df_movies.shape

(1681, 25)

In [224]:
df_movies = df_movies.dropna(subset=["fecha"])

In [226]:
movie_embeddings_matrix.shape

(1683, 6)

In [227]:
user_embeddings_matrix.shape

(944, 6)

In [228]:
pd.DataFrame(user_embeddings_matrix).describe()

,0,1,2,3,4,5
count,9.440000e+02,9.440000e+02,9.440000e+02,9.440000e+02,9.440000e+02,9.440000e+02
mean,-5.531942e-01,5.451787e-01,-5.485067e-01,5.530015e-01,5.526607e-01,5.415881e-01
std,1.763010e-01,1.734075e-01,1.745770e-01,1.763287e-01,1.759321e-01,1.729536e-01
min,-9.954566e-01,-1.983282e-33,-9.865357e-01,-3.425232e-33,3.199355e-33,-1.851279e-33
25%,-6.957088e-01,4.189921e-01,-6.894269e-01,4.247965e-01,4.245950e-01,4.157251e-01
50%,-5.595111e-01,5.515438e-01,-5.548432e-01,5.592859e-01,5.590334e-01,5.475787e-01
75%,-4.249332e-01,6.850068e-01,-4.214294e-01,6.956081e-01,6.946417e-01,6.817395e-01
max,-4.604195e-33,9.805635e-01,3.069597e-33,9.952666e-01,9.943194e-01,9.751592e-01


## Nos conectamos a la Base de datos

In [126]:
host = 'localhost'
port = 9200
auth = ('admin', 'admin')

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
)

c:\Users\josez\.conda\envs\TP2\lib\site-packages\opensearchpy\connection\http_urllib3.py:199: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [229]:
stats = client.ping()
print(stats)

True


C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [230]:
indices = client.indices.get_alias("*")
for index in indices:
    print(index) 

october_2
october_1
.opensearch-sap-pre-packaged-rules-config
october_4
movie
october_3
october_6
security-auditlog-2023.09.01
october_5
october_8
october_7
security-auditlog-2023.10.23
october_9
security-auditlog-2023.10.24
security-auditlog-2023.10.18
.opendistro_security
security-auditlog-2023.10.19
security-auditlog-2023.10.17
.opensearch-observability
.plugins-ml-config
october_10
.opensearch-sap-log-types-config


C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [231]:
try:
     client.indices.delete('movie')
except Exception as e:
    print(str(e))

C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [232]:
method = {
    "name": "hnsw",
    "space_type": "cosinesimil",
    "engine": "nmslib"
}

In [233]:
class KNNVector(Field):
    name = "knn_vector"
    def __init__(self, dimension, method, **kwargs):
        super(KNNVector, self).__init__(dimension=dimension, method=method, **kwargs)


## Defino la clase para guardar en Opensearch

In [234]:
index_name = 'movie'
class Movie(Document):
    movie_id = Keyword()
    name = Keyword()
    release_date = Text()
    url = Text()
    action = Boolean 
    adventure = Boolean()
    animation = Boolean()
    children = Boolean()
    comedy = Boolean()
    crime = Boolean()
    documentary = Boolean()
    drama = Boolean()
    fantasy = Boolean()
    film_noir = Boolean()
    horror = Boolean()
    musical = Boolean()
    mystery = Boolean()
    romance = Boolean()
    sci_fi = Boolean()
    thriller = Boolean()
    war = Boolean()
    western = Boolean()
    terror = Boolean()
    created_at = Date()

    vector = KNNVector(
        movie_embeddings_matrix.shape[1],
        method
    )
    class Index:
        name = index_name
        settings = {
                'index': {
                'knn': True
            }
        }

    def save(self, ** kwargs):
        self.meta.id = self.movie_id
        return super(Movie, self).save(** kwargs)

In [235]:
Movie.init(using=client)

C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [236]:
client.indices.get(index="mo*")

C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'movie': {'aliases': {},
  'mappings': {'properties': {'adventure': {'type': 'boolean'},
    'animation': {'type': 'boolean'},
    'children': {'type': 'boolean'},
    'comedy': {'type': 'boolean'},
    'created_at': {'type': 'date'},
    'crime': {'type': 'boolean'},
    'documentary': {'type': 'boolean'},
    'drama': {'type': 'boolean'},
    'fantasy': {'type': 'boolean'},
    'film_noir': {'type': 'boolean'},
    'horror': {'type': 'boolean'},
    'movie_id': {'type': 'keyword'},
    'musical': {'type': 'boolean'},
    'mystery': {'type': 'boolean'},
    'name': {'type': 'keyword'},
    'release_date': {'type': 'text'},
    'romance': {'type': 'boolean'},
    'sci_fi': {'type': 'boolean'},
    'terror': {'type': 'boolean'},
    'thriller': {'type': 'boolean'},
    'url': {'type': 'text'},
    'vector': {'type': 'knn_vector',
     'dimension': 6,
     'method': {'engine': 'nmslib',
      'space_type': 'cosinesimil',
      'name': 'hnsw',
      'parameters': {}}},
    'war': {'type'

In [237]:
client.indices.exists('movie')

C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


True

In [238]:
df_movies.head(2)

,id,name,fecha,x,url,unknown,Action,Adventure,Animation,Children's,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movieIdx
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,25
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,148


In [239]:
for i , row in df_movies.iterrows():
    #print(row['id'], row['name'], row['fecha'], row['url'], (bool)(row['Action']), list(movie_embeddings_matrix[row.movieIdx]))
    mv = Movie(
        movie_id = row['id'],
        name = row['name'],
        release_date = row['fecha'],
        url = row['url'],
        action =  (bool)(row['Action']),
        adventure = (bool)(row['Adventure']),
        animation = (bool)(row['Animation']),
        children = (bool)(row["Children's"]),
        comedy = (bool)(row['Comedy']),
        crime = (bool)(row['Crime']),
        documentary = (bool)(row['Documentary']),
        drama = (bool)(row['Drama']),
        fantasy = (bool)(row['Fantasy']),
        film_noir = (bool)(row['Film-Noir']),
        horror = (bool)(row['Horror']),
        musical = (bool)(row['Musical']),
        mystery = (bool)(row['Mystery']),
        romance = (bool)(row['Romance']),
        sci_fi = (bool)(row['Sci-Fi']),
        thriller = (bool)(row['Thriller']),
        war = (bool)(row['War']),
        western = (bool)(row['Western']),
        created_at = datetime.datetime.now(),
        vector = list(movie_embeddings_matrix[row.movieIdx])
    )
    print(row['name'], row['id'])
    mv.save(using=client)

#    if i == 5:
#        break


Toy Story (1995) 1


C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


GoldenEye (1995) 2
Four Rooms (1995) 3
Get Shorty (1995) 4
Copycat (1995) 5
Shanghai Triad (Yao a yao yao dao waipo qiao) (1995) 6


C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\josez\AppData

Twelve Monkeys (1995) 7
Babe (1995) 8
Dead Man Walking (1995) 9
Richard III (1995) 10


...

C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\josez\AppData

Sunchaser, The (1996) 1675
War at Home, The (1996) 1676
Sweet Nothing (1995) 1677
Mat' i syn (1997) 1678


C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\josez\AppData

B. Monkey (1998) 1679
Sliding Doors (1998) 1680
You So Crazy (1994) 1681
Scream of Stone (Schrei aus Stein) (1991) 1682


C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\josez\AppData

In [240]:
indices = client.indices.get_alias('movie')
indices

C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'movie': {'aliases': {}}}

In [242]:
index = "movie"
try:
    response = client.search(index=index)
    for h in response['hits']['hits']:
       print(h)
       #print(h['_source']['vector'], h['_source']['movie_id'])
       
except Exception as e:
    print(str(e))

{'_index': 'movie', '_id': '1', '_score': 1.0, '_source': {'movie_id': 1, 'name': 'Toy Story (1995)', 'release_date': '01-Jan-1995', 'url': 'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)', 'action': False, 'adventure': False, 'animation': True, 'children': True, 'comedy': True, 'crime': False, 'documentary': False, 'drama': False, 'fantasy': False, 'film_noir': False, 'horror': False, 'musical': False, 'mystery': False, 'romance': False, 'sci_fi': False, 'thriller': False, 'war': False, 'western': False, 'created_at': '2023-10-24T15:12:22.534576', 'vector': [-0.9113113880157471, 0.899655818939209, -0.9045496582984924, 0.9107000827789307, 0.9114372134208679, 0.8904204368591309]}}
{'_index': 'movie', '_id': '2', '_score': 1.0, '_source': {'movie_id': 2, 'name': 'GoldenEye (1995)', 'release_date': '01-Jan-1995', 'url': 'http://us.imdb.com/M/title-exact?GoldenEye%20(1995)', 'action': True, 'adventure': True, 'animation': False, 'children': False, 'comedy': False, 'crime': False, 'd

C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [243]:
Movie.search(using=client).count()

C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


1681

## Buscamos la pelicula con movie_id = 5

In [244]:
movie_idx_to_search = 5

In [245]:
df_movies[df_movies['movieIdx'] == movie_idx_to_search]

,id,name,fecha,x,url,unknown,Action,Adventure,Animation,Children's,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movieIdx
345,346,Jackie Brown (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?imdb-title-11...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [246]:
movie_embeddings_matrix[movie_idx_to_search]

array([-0.73561245,  0.7251868 , -0.72959447,  0.7352088 ,  0.7350541 ,
        0.7197288 ], dtype=float32)

## Obtenemos el vector de embedding correspondiente de la pelicula, de la DB OpenSearch

In [247]:
query = {
    "size": 5,
    "query": {
        "knn": {
        "vector": {
            "vector": movie_embeddings_matrix[movie_idx_to_search],
            "k" : 20
        }
        }
    }
}

In [248]:
response = client.search(index='movie', body=query)

C:\Users\josez\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [251]:
for h in response['hits']['hits']:
    print(h['_source']['movie_id'], h['_source']['name'])


9 Dead Man Walking (1995)
22 Braveheart (1995)
24 Rumble in the Bronx (1995)
64 Shawshank Redemption, The (1994)
66 While You Were Sleeping (1995)
